In [28]:
import cfscrape
from lxml import etree
import pandas as pd
scraper = cfscrape.create_scraper()
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [3]:
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

# Retrieving MITRE ATT&CK Enterprise

In [32]:
url="https://attack.mitre.org/tactics/enterprise/"
scraped_html=scraper.get(url,headers=header).content
tactics_enterprise = pd.read_html(scraped_html)[0]
tactics_enterprise = tactics_enterprise.rename(columns = {'ID':'tactic_id', 'Name':'tactic_name','Description':'tactic_description'})
tactics_enterprise

,tactic_id,tactic_name,tactic_description
0,TA0001,Initial Access,The adversary is trying to get into your network.
1,TA0002,Execution,The adversary is trying to run malicious code.
2,TA0003,Persistence,The adversary is trying to maintain their foothold.
3,TA0004,Privilege Escalation,The adversary is trying to gain higher-level permissions.
4,TA0005,Defense Evasion,The adversary is trying to avoid being detected.
5,TA0006,Credential Access,The adversary is trying to steal account names and passwords.
6,TA0007,Discovery,The adversary is trying to figure out your environment.
7,TA0008,Lateral Movement,The adversary is trying to move through your environment.
8,TA0009,Collection,The adversary is trying to gather data of interest to their goal.
9,TA0011,Command and Control,The adversary is trying to communicate with compromised systems to control them.


In [33]:
df_enterprise_tnt = pd.DataFrame()

In [34]:
for tactic_id in tactics_enterprise['tactic_id'].values:
    url="https://attack.mitre.org/tactics/"+tactic_id
    scraped_html=scraper.get(url,headers=header).content
    df_temp = pd.read_html(scraped_html)[0]
    df_temp = df_temp.rename(columns = {'ID':'technique_id', 'Name':'technique_name','Description':'technique_description'})
    df_temp['tactic_id'] = tactic_id
    df_enterprise_tnt = df_enterprise_tnt.append(df_temp)

In [87]:
all_tactics_techniques_enterprise = pd.merge(df_enterprise_tnt, tactics_enterprise, how='inner', on='tactic_id')

# Retrieving MITRE ATT&CK Enterprise Cloud (from a matrix)

In [72]:
url='https://attack.mitre.org/matrices/enterprise/cloud/'
scraped_html=scraper.get(url,headers=header).content
# print(scraped_html.decode("utf-8"))

In [73]:
html = etree.HTML(scraped_html)
elements = html.xpath("//a[contains(@href,'techniques/T')]/@href");

### Getting the IDs of techniques related to the Cloud matrix

In [94]:
techniques = pd.DataFrame(elements)[0].str.replace('/techniques/','')

### Correlating the techniques in the Cloud with the generic techniques in Enterprises

In [93]:
cloud_tactics_techniques = all_tactics_techniques_enterprise[all_tactics_techniques_enterprise['technique_id'].isin(techniques)]
cloud_tactics_techniques

,technique_id,technique_name,technique_description,tactic_id,tactic_name,tactic_description
0,T1189,Drive-by Compromise,"A drive-by compromise is when an adversary gains access to a system through a user visiting a website over the normal course of browsing. With this technique, the user's web browser is typically targeted for exploitation, but adversaries may also use compromised websites for non-exploitation behavior such as acquiring application access tokens.",TA0001,Initial Access,The adversary is trying to get into your network.
1,T1190,Exploit Public-Facing Application,"The use of software, data, or commands to take advantage of a weakness in an Internet-facing computer system or program in order to cause unintended or unanticipated behavior. The weakness in the system can be a bug, a glitch, or a design vulnerability. These applications are often websites, but can include databases (like SQL), standard services (like SMB or SSH), and any other applications with Internet accessible open sockets, such as web servers and related services. Depending on the flaw being exploited this may include Exploitation for Defense Evasion.",TA0001,Initial Access,The adversary is trying to get into your network.
6,T1192,Spearphishing Link,"Spearphishing with a link is a specific variant of spearphishing. It is different from other forms of spearphishing in that it employs the use of links to download malware contained in email, instead of attaching malicious files to the email itself, to avoid defenses that may inspect email attachments.",TA0001,Initial Access,The adversary is trying to get into your network.
9,T1199,Trusted Relationship,Adversaries may breach or otherwise leverage organizations who have access to intended victims. Access through trusted third party relationship exploits an existing connection that may not be protected or receives less scrutiny than standard mechanisms of gaining access to a network.,TA0001,Initial Access,The adversary is trying to get into your network.
10,T1078,Valid Accounts,Adversaries may steal the credentials of a specific user or service account using Credential Access techniques or capture credentials earlier in their reconnaissance process through social engineering for means of gaining Initial Access.,TA0001,Initial Access,The adversary is trying to get into your network.
47,T1098,Account Manipulation,"Account manipulation may aid adversaries in maintaining access to credentials and certain permission levels within an environment. Manipulation could consist of modifying permissions, modifying credentials, adding or changing permission groups, modifying account settings, or modifying how authentication is performed. These actions could also include account activity designed to subvert security policies, such as performing iterative password updates to subvert password duration policies and preserve the life of compromised credentials. In order to create or manipulate accounts, the adversary must already have sufficient permissions on systems or the domain.",TA0003,Persistence,The adversary is trying to maintain their foothold.
58,T1136,Create Account,"Adversaries with a sufficient level of access may create a local system, domain, or cloud tenant account. Such accounts may be used for persistence that do not require persistent remote access tools to be deployed on the system.",TA0003,Persistence,The adversary is trying to maintain their foothold.
68,T1525,Implant Container Image,"Amazon Web Service (AWS) Amazon Machine Images (AMI), Google Cloud Platform (GCP) Images, and Azure Images as well as popular container runtimes such as Docker can be implanted or backdoored to include malicious code. Depending on how the infrastructure is provisioned, this could provide persistent access if the infrastructure provisioning tool is instructed to always use the latest image.",TA0003,Persistence,The adversary is trying to maintain their foothold.
81,T1137,Office Application Startup,Microsoft Office is a f

In [95]:
len(cloud_tactics_techniques)

43

### Export to .CSV

In [96]:
cloud_tactics_techniques.to_csv('mitre_attack_tactics_n_techniques_cloud.csv')